In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
df_order_historic_demand = pd.read_excel('C:/Users/JWinn01/Desktop/IABD/material/datos/Datos_P1/df_historic_order_demand.xlsx')
distances = pd.read_excel('C:/Users/JWinn01/Desktop/IABD/material/datos/Datos_P1/df_distance_km.xlsx')
df_distance_min = pd.read_excel('C:/Users/JWinn01/Desktop/IABD/material/datos/Datos_P1/df_distance_min.xlsx')
df_location = pd.read_excel('C:/Users/JWinn01/Desktop/IABD/material/datos/Datos_P1/df_location.xlsx')
customers_df = pd.read_excel('C:/Users/JWinn01/Desktop/IABD/material/datos/Datos_P1/df_orders.xlsx')
vehicles_df = pd.read_excel('C:/Users/JWinn01/Desktop/IABD/material/datos/Datos_P1/df_vehicle.xlsx')

In [9]:
#cambiar posicion de almacen por mas comodidad

# Mostrar las primeras filas para verificar el contenido original
print("Datos originales:")
print(distances.head())


Datos originales:
   Cliente_1  Cliente_2  Cliente_3  Cliente_4  Cliente_5  Cliente_6  \
0     0.0000     7.5625    15.5365     1.1998     4.7145     1.7407   
1     7.5625     0.0000     3.3838     7.7433    14.5720     8.5237   
2    15.5365     3.3838     0.0000    12.5438     0.0000     0.0000   
3     1.1998     7.7433    12.5438     0.0000     5.0721     0.9119   
4     4.7145    14.5720     0.0000     5.0721     0.0000     4.8187   

   Cliente_7  Cliente_8  Cliente_9  Cliente_10  ...  Cliente_12  Cliente_13  \
0     7.9408    17.1947     4.2933      3.2659  ...      6.0225      5.4470   
1     0.4847    13.7974    10.1522      7.1521  ...     10.1049      2.6961   
2     0.0000    16.0355    13.9120     13.0649  ...     12.3430      5.0114   
3     7.5798    17.4095     3.5781      3.3451  ...      6.2330      4.7117   
4     0.0000     0.0000     0.0000      7.2170  ...      6.8738      9.1064   

   Cliente_14  Cliente_15  Cliente_16  Cliente_17  Cliente_18  Cliente_19  \
0  

In [5]:
# Parámetros de ACO
n_ants = 10
n_iterations = 100
alpha = 1.0
beta = 2.0
rho = 0.5
Q = 100

n_nodes = len(customers_df)+1
pheromone_matrix = np.ones((n_nodes, n_nodes))

def calculate_probabilities(current_node, unvisited_nodes, pheromones, distances):
    probabilities = []
    for node in unvisited_nodes:
        distance = distances.iloc[current_node, node]
        if distance > 0:  
            tau = pheromones[current_node][node] ** alpha
            eta = (1 / distance) ** beta
            probabilities.append(tau * eta)
        else:
            probabilities.append(0)  
    
    probabilities = np.array(probabilities)
    total = probabilities.sum()
    
    # Evitar misma posicion entre clientes (DUDA)
    if total == 0:
        probabilities = np.ones(len(unvisited_nodes)) / len(unvisited_nodes)
    else:
        probabilities = probabilities / total
    
    return probabilities

# Función para construir solución por una hormiga
def construct_solution_with_vehicle(pheromones, distances, vehicles_df, orders):
    solution = []  
    remaining_customers = set(range(n_nodes - 1))  
    depot = n_nodes - 1  # indice del depósito

    for _, vehicle in vehicles_df.iterrows():  
        vehicle_routes = []  
        vehicle_capacity = vehicle["capacidad_kg"]

        while remaining_customers:
            current_route = [depot]  # Inicia en el depósito
            current_capacity = vehicle_capacity
            current_node = depot
            while remaining_customers:
                unvisited_nodes = [node for node in remaining_customers if orders[node] <= current_capacity]
                if not unvisited_nodes:  # No hay clientes factibles
                    break

                probabilities = calculate_probabilities(current_node, unvisited_nodes, pheromone_matrix, distances)
                next_node = np.random.choice(unvisited_nodes, p=probabilities)

                current_route.append(next_node)
                current_capacity -= orders[next_node]
                remaining_customers.remove(next_node)
                current_node = next_node

            current_route.append(depot) 
            if len(current_route) > 2: 
                vehicle_routes.append(current_route)

        if vehicle_routes:  
            solution.append((vehicle["vehiculo_id"], vehicle_routes))
    return solution



# costo por vehiculo
def calculate_solution_cost(solution, distances, vehicles_df):
    total_cost = 0
    vehicle_costs = []
    for vehicle_name, routes in solution:
        vehicle = vehicles_df.loc[vehicles_df["vehiculo_id"] == vehicle_name]
        price_per_km = vehicle["costo_km"].values[0]
        vehicle_total = 0
        
        for route in routes:
            route_cost = sum(distances.iloc[route[i], route[i + 1]] for i in range(len(route) - 1))
            vehicle_total += route_cost * price_per_km
        total_cost += vehicle_total
        vehicle_costs.append((vehicle_name, vehicle_total))
    return total_cost, vehicle_costs


def construct_solution(pheromones, distances, vehicle_capacity, orders):
    solution = []
    remaining_customers = set(range(1, n_nodes))  
    while remaining_customers:
        current_route = [0]  # "Decimos aqui desde donde empezamos" por ende decimos el almacen
        current_capacity = vehicle_capacity
        current_node = 0

        while remaining_customers:
            unvisited_nodes = [node for node in remaining_customers if orders[node] <= current_capacity]
            if not unvisited_nodes:  # Evitar excepciones
                break

            # Llamada a funcion T2
            probabilities = calculate_probabilities(current_node, unvisited_nodes, pheromones, distances)
            next_node = np.random.choice(unvisited_nodes, p=probabilities)
            
            current_route.append(next_node)
            current_capacity -= orders[next_node]
            remaining_customers.remove(next_node)
            current_node = next_node

        current_route.append(0)  # regresamos hacia el almacen
        solution.append(current_route)
    return solution


# costo de una solución
def calculate_cost(solution, distances, price_per_km):
    total_cost = 0
    for route in solution:
        route_cost = sum(distances.iloc[route[i], route[i + 1]] for i in range(len(route) - 1))
        total_cost += route_cost * price_per_km
    return total_cost





In [ ]:
# Output
best_solution = None
best_cost = float('inf')

for iteration in range(n_iterations):
    solutions = []
    costs = []

    #Soluciones distintas, revisar para posibles mejores 
    for _ in range(n_ants):
        solution = construct_solution_with_vehicle(pheromone_matrix, distances, vehicles_df, customers_df["order_demand"])
        total_cost, vehicle_costs = calculate_solution_cost(solution, distances, vehicles_df)
        solutions.append(solution)
        costs.append(total_cost)

        # Actualizar mejor solución
        if total_cost < best_cost:
            best_cost = total_cost
            best_solution = solution

    pheromone_matrix *= (1 - rho)
    for solution, cost in zip(solutions, costs):
        for vehicle_name, routes in solution:
            for route in routes:
                for i in range(len(route) - 1):
                    pheromone_matrix[route[i]][route[i + 1]] += Q / cost

customers_df = pd.concat( #Añadimos esto(error de indice si quitamos esto)
    [pd.DataFrame({"Cliente": ["Almacén"], "order_demand": [0]}), customers_df],
    ignore_index=True
)
print("\nMejor solución encontrada:")
for vehicle_name, routes in best_solution:
    print(f"Vehículo: {vehicle_name}")
    for i, route in enumerate(routes):
        print(customers_df.iloc)
        clientes = [customers_df.iloc[node]['cliente'] for node in route]
        print(f"  Ruta {i + 1}: {clientes}")
print(f"Costo total: {best_cost}")

Iteración 1: Mejor costo = 47.335620000000006
Iteración 2: Mejor costo = 47.33562
Iteración 3: Mejor costo = 47.33562
Iteración 4: Mejor costo = 47.33562
Iteración 5: Mejor costo = 47.33562
Iteración 6: Mejor costo = 47.33562
Iteración 7: Mejor costo = 47.33562
Iteración 8: Mejor costo = 47.33562
Iteración 9: Mejor costo = 47.33562
Iteración 10: Mejor costo = 47.33562
Iteración 11: Mejor costo = 47.33562
Iteración 12: Mejor costo = 47.33562
Iteración 13: Mejor costo = 47.33562
Iteración 14: Mejor costo = 47.33562
Iteración 15: Mejor costo = 47.33562
Iteración 16: Mejor costo = 47.33562
Iteración 17: Mejor costo = 47.33562
Iteración 18: Mejor costo = 47.33562
Iteración 19: Mejor costo = 47.33562
Iteración 20: Mejor costo = 47.33562
Iteración 21: Mejor costo = 47.33562
Iteración 22: Mejor costo = 47.33562
Iteración 23: Mejor costo = 47.33562
Iteración 24: Mejor costo = 47.33562
Iteración 25: Mejor costo = 47.33562
Iteración 26: Mejor costo = 47.33562
Iteración 27: Mejor costo = 47.33562
